In [1]:
import warnings
warnings.filterwarnings("ignore") 

from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt()
client = Client(cluster)
cluster

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import gcsfs # Pythonic file-system for Google Cloud Storage
import os.path
import xesmf as xe
from scipy.signal import detrend
from scipy.optimize import curve_fit
import turtle

import os
os.environ['NUMPY_EXPERIMENTAL_ARRAY_FUNCTION'] = '0'

In [2]:
import copy
def get_data(df, var, model, expe, freq):
    try:
        uri = df[(df.variable_id == var) & \
                 (df.source_id == model) & \
                 (df.experiment_id == expe) & \
                 (df.table_id == freq)].zstore.values[0]
        gcs = gcsfs.GCSFileSystem(token='anon')
        ds = xr.open_zarr(gcs.get_mapper(uri), consolidated=True)
#         print(model, var, "found data")
    except:
        ds = []
        #print(model, var, "no data")
    return ds        

df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
models = ['CESM2', 'CESM2-WACCM', 'GFDL-CM4',
         'GFDL-ESM4', 'IPSL-CM6A-LR', 'GISS-E2-1-G',
         'GISS-E2-1-G-CC', 'MIROC-ES2L', 'NorCPM1',
         'NorESM2-LM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR',
         'UKESM1-0-LL', 'CNRM-ESM2-1', 'ACCESS-ESM1-5',
         'CanESM5-CanOE', 'CanESM5', 'EC-Earth3']

ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 91, 1.0)),
                     'lon': (['lon'], np.arange(0, 361, 1.0)),
                    }
                   )
ds_out

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 361)
Coordinates:
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0 360.0
Data variables:
    *empty*

In [3]:
expe = 'historical'
freq = 'Omon'
start_year = '1991'
end_year = '2010'
var = 'zooc'
model = 'MPI-ESM1-2-HR'
vvar = var + 'os'
xray = get_data(df, vvar, model, expe, freq)
print(xray)
print('xray above')
if isinstance(xray, xr.Dataset):
        xxray_in = xray
        print('was')
else:
        vvar = var    
        xray = get_data(df, vvar, model, expe, freq)
        xxray_in= xray

[]
xray above


### Access data

In [4]:
# functions for fetching cmip6 data from google cloud
def rename_coords(ds):
    ds = ds.copy()
    """Rename all depth dim to `lev`"""
    if "olevel" in ds.coords:
        ds = ds.rename({"olevel": "lev"})
    if "lev_partial" in ds.coords:
        ds = ds.rename({"lev_partial": "lev"})
    """Rename all latitude, longitude dim to `lat`,`lon`"""
    if 'latitude' in ds.coords:
        ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
    if 'nav_lat' in ds.coords:
        ds = ds.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
    return ds

def get_dataset(col, var, freq, expe, model, grid):
    dataset = col.search(variable_id = var, table_id = freq, experiment_id = expe, 
                         source_id = model, grid_label = grid).to_dataset_dict(
        zarr_kwargs= {'consolidated': True, 'decode_times':True}, preprocess = rename_coords)
    dataset = dataset[list(dataset)[0]].squeeze('member_id').reset_coords('member_id', drop = True)
    return dataset

The data we are using here is from model GFDL-CM4's piControl data. We are using monthly ocean data and the variables we are looking for is 'so' sanility and 'thetao' temperature.

In [5]:
model_name = 'GFDL-CM4' 
experiment = 'piControl'
frequency = 'Omon'
variables = ['so', 'thetao']

In [6]:
# 'gr' is the regrided data (with one-dimentional lon & lat)
# dataset_gr = get_dataset(col, variables, frequency, experiment, model_name, 'gr')

In [7]:
def smow(t):
    a = (999.842594, 6.793952e-2, -9.095290e-3, 1.001685e-4, -1.120083e-6,
         6.536332e-9)
    T68 = t * 1.00024
    return (a[0] + (a[1] + (a[2] + (a[3] + (a[4] + a[5] * T68) * T68) * T68) *
            T68) * T68)

def dens0(s, t):
    T68 = t * 1.00024
    b = (8.24493e-1, -4.0899e-3, 7.6438e-5, -8.2467e-7, 5.3875e-9)
    c = (-5.72466e-3, 1.0227e-4, -1.6546e-6)
    d = 4.8314e-4
    return (smow(t) + (b[0] + (b[1] + (b[2] + (b[3] + b[4] * T68) * T68) *
            T68) * T68) * s + (c[0] + (c[1] + c[2] * T68) * T68) * s *
            s ** 0.5 + d * s ** 2)

def func_mld(dens_diff, depths):
    '''
    Function for calculate mld from density difference (den - den10 - 0.03) and depth
    Return mixed layer depth 
    '''
    if np.isnan(dens_diff[0]):
        mld = np.nan
    elif dens_diff[0] >= 0:
        mld = np.nan
    else:
        nthr_index = np.where(dens_diff > 0)[0]
        if len(nthr_index) == 0:
            naninds = np.where(np.isnan(dens_diff))[0]
            if len(naninds) > 0:
                nanindex = naninds[0]
            else:
                nanindex = len(depths)
            mld = depths[nanindex-1]
        else:
            nind = nthr_index[0] + 1
            mld = np.interp(0, dens_diff[nind-2:nind], depths[nind-2:nind])                
    return mld

def xr_func_mld(dens):
    '''
    Function for parallel computing
    '''
    dens10 = dens.interp(lev = 10, method = 'linear')  # density at 10m
    dens_diff = dens - dens10 - 0.03               # density differences 
    mld = xr.apply_ufunc(
        func_mld, 
        dens_diff,#.chunk({"time":25, "x":30, "y":30}),  
        dens_diff.lev, 
        input_core_dims = [["lev"], ["lev"]], 
        vectorize = True,
        dask = "parallelized",
        output_dtypes = [dens_diff.lev.dtype],
    )
    return mld

In [8]:
models = ['CESM2-WACCM','GFDL-CM4','GFDL-ESM4','IPSL-CM6A-LR','GISS-E2-1-G','GISS-E2-1-G-CC','MIROC-ES2L',
          'NorCPM1', 'NorESM2-LM','MPI-ESM1-2-HR','MPI-ESM1-2-LR','UKESM1-0-LL','CNRM-ESM2-1','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5', 
          'EC-Earth3']
# models = ['CESM2-WACCM']
var_names = ['so','thetao']

xxray_in = []
for model in models:
    print(model)
    check1 = False
    check2 = False
    for var in var_names:
        vvar = var + 'os'
        xray = get_data(df, vvar, model, expe, freq)
        if isinstance(xray, xr.Dataset):
            xxray_in = xray
            
        else:
            vvar = var    
            xray = get_data(df, vvar, model, expe, freq)
            xxray_in= xray
            
#         if model == 'MPI-ESM1-2-HR' and (var == 'zooc' or var == 'si'):
        if 1==2:
            print('wont work')
        else:
            if isinstance(xxray_in, xr.Dataset):
                    ds = xxray_in.sel(time=slice(start_year, end_year))[vvar]
                    
                    if 'latitude' in ds.coords:
                        ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
                    if 'nav_lat' in ds.coords:
                        ds = ds.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
                    
                    if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                        if var == 'phyc' or var == 'chl' or var == 'phydiat' or var == 'zooc' or var == 'no3' or var == 'si' or var == 'dfe':
                            ds = ds.where(ds>=0)
                            
                    ds_in = xr.Dataset({"lat": ds.lat, "lon": ds.lon})
                    dsr = xe.Regridder(ds_in, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
                    dsr._grid_in = None
                    dsr._grid_out = None
                    dsr_out0 = dsr(ds)
                    dsr_out = xr.Dataset({var:dsr_out0})
                    if var == 'so':
                        salt = dsr_out
                        check1=True
                    if var == 'thetao':
                        temp = dsr_out
                        check2=True
    mld = xr_func_mld(dens0(salt.so,temp.thetao))
    dsr_out = mld
    var='mld'
    if dsr_out['time'].shape[0]==12*20: #5 years 12 months
        print(var)

        ofn = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_np.txt'
        if(os.path.exists(ofn)):
        # if 1==2:
            print('already calculated')
        else:
            print('calculating')
            ds_np = dsr_out.sel(lat=slice(45,50)).sel(lon=slice(210,220)) #45 north, 210 east
            dsm_np = ds_np.mean(dim='time',skipna=1)
            anom_np = ds_np - dsm_np
            anom_np.load()
#                     anom_detrended_np = xr.apply_ufunc(detrend, anom_np.fillna(0), anom_np.chunk({"lon":45,"lat":45}), kwargs={'axis': 0}).where(~anom_np.isnull())
            anom_detrended_np = xr.apply_ufunc(detrend, anom_np.fillna(0).chunk({"lon":45,"lat":45}), kwargs={'axis': 0},dask="parallelized", output_dtypes=[anom_np.dtype]).where(~anom_np.isnull())
            detrend_np = anom_detrended_np + dsm_np
            do_np = detrend_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values

            if var == 'spco2':
                do_np = do_np / 0.101325
#                         if var == 'phyc' or var == 'phycos':
#                             do_np = do_np * 12000 # more info
#                         if var == 'phycdiat'or var == 'phycdiatos':
#                             do_np = do_np * 12000 # more info
            if var == 'fgco2':
                do_np = do_np * 3600 * 24 * 365 * 1000 / (44/12) 
            
            savef_np = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_np.txt'
            with open(savef_np, 'w') as npf:
                for idata in do_np:
                    npf.write(str(idata) +"\n")

        ofn = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_na.txt'
        if(os.path.exists(ofn)):
        # if 1==2:
            print('already calculated')
        else:
            print('calculating')
            ds_na = dsr_out.sel(lat=slice(45,50)).sel(lon=slice(325,335))
            dsm_na = ds_na.mean(dim='time',skipna=1)
            anom_na = ds_na - dsm_na
            anom_na.load()
#                     anom_detrended_np = xr.apply_ufunc(detrend, anom_np.fillna(0), anom_np.chunk({"lon":45,"lat":45}), kwargs={'axis': 0}).where(~anom_np.isnull())
            anom_detrended_na = xr.apply_ufunc(detrend, anom_na.fillna(0).chunk({"lon":45,"lat":45}), kwargs={'axis': 0},dask="parallelized", output_dtypes=[anom_na.dtype]).where(~anom_na.isnull())
            detrend_na = anom_detrended_na + dsm_na
            do_na = detrend_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values

            if var == 'spco2':
                do_na = do_na / 0.101325
#                         if var == 'phyc' or var == 'phycos':
#                             do_np = do_np * 12000 # more info
#                         if var == 'phycdiat'or var == 'phycdiatos':
#                             do_np = do_np * 12000 # more info
            if var == 'fgco2':
                do_na = do_na * 3600 * 24 * 365 * 1000 / (44/12) 

            savef_na = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_na.txt'
            with open(savef_na, 'w') as naf:
                for idata in do_na:
                    naf.write(str(idata) +"\n")

        ofn = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_new.txt'
        if(os.path.exists(ofn)):
        # if 1==2:
            print('already calculated')
        else:
            print('calculating')
            ds_new = dsr_out.sel(lat=slice(45,50)).sel(lon=slice(342,347))
            dsm_new = ds_new.mean(dim='time',skipna=1)
            anom_new = ds_new - dsm_new
            anom_new.load()
#                     anom_detrended_np = xr.apply_ufunc(detrend, anom_np.fillna(0), anom_np.chunk({"lon":45,"lat":45}), kwargs={'axis': 0}).where(~anom_np.isnull())
            anom_detrended_new = xr.apply_ufunc(detrend, anom_new.fillna(0).chunk({"lon":45,"lat":45}), kwargs={'axis': 0},dask="parallelized", output_dtypes=[anom_new.dtype]).where(~anom_new.isnull())
            detrend_new = anom_detrended_new + dsm_new
            do_new = detrend_new.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values

            if var == 'spco2':
                do_new = do_new / 0.101325
#                         if var == 'phyc' or var == 'phycos':
#                             do_np = do_np * 12000 # more info
#                         if var == 'phycdiat'or var == 'phycdiatos':
#                             do_np = do_np * 12000 # more info
            if var == 'fgco2':
                do_new = do_new * 3600 * 24 * 365 * 1000 / (44/12) 

            savef_new = '/home/jovyan/tempfolder/' + model + '_' + var + start_year + end_year + '_new.txt'
            with open(savef_new, 'w') as newf:
                for idata in do_new:
                    newf.write(str(idata) +"\n")

# #                 dsr.clean_weight_file()

# #                 :~$ tar -czvf NA_NPssp585.tar.gz save

CESM2-WACCM
mld
already calculated
already calculated
already calculated
GFDL-CM4
mld
calculating


KilledWorker: ("('open_dataset-getitem-45622b80bef97add069ef84f0edf5f54', 147, 0, 0, 0)", <WorkerState 'tls://10.8.13.6:37653', name: dask-worker-325db5f39f864973a1690980545dc183-hxdmz, status: closed, memory: 0, processing: 9>)

### Calculate MLD
We want to calculate the mid layer depth (MLD) from the temperature and sanility we've got.

In [11]:
anom_np.load()

LZ4BlockError: Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 9

In [ ]:
# functions for calculating potential density of selected year & month from temp & sanility.
# def sel_time(ds, start_year, end_year, month = None):
#     ds = ds.isel(time = slice((start_year-1)*12, end_year*12))
#     if month:
#         ds = list(ds.groupby("time.month"))[month-1][-1]
#     return ds

def smow(t):
    a = (999.842594, 6.793952e-2, -9.095290e-3, 1.001685e-4, -1.120083e-6,
         6.536332e-9)
    T68 = t * 1.00024
    return (a[0] + (a[1] + (a[2] + (a[3] + (a[4] + a[5] * T68) * T68) * T68) *
            T68) * T68)

def dens0(s, t):
    T68 = t * 1.00024
    b = (8.24493e-1, -4.0899e-3, 7.6438e-5, -8.2467e-7, 5.3875e-9)
    c = (-5.72466e-3, 1.0227e-4, -1.6546e-6)
    d = 4.8314e-4
    return (smow(t) + (b[0] + (b[1] + (b[2] + (b[3] + b[4] * T68) * T68) *
            T68) * T68) * s + (c[0] + (c[1] + c[2] * T68) * T68) * s *
            s ** 0.5 + d * s ** 2)

# def func_dens0(dataset, start_year, end_year, month_no = 9): #
#     da_t = sel_time(dataset.thetao, start_year, end_year, month = month_no)
#     da_s = sel_time(dataset.so, start_year, end_year, month = month_no)
#     da_dens = dens0(da_s, da_t)
#     return da_dens

In [ ]:
# da_den = dens0(da_s, da_t)

In [ ]:
# functions for calculating MLD
# def func_mld(dens_diff, depths):
#     '''
#     Function for calculate mld from density difference (den - den10 - 0.03) and depth
#     Return mixed layer depth 
#     '''
#     if np.isnan(dens_diff[0]):
#         mld = np.nan
#     elif dens_diff[0] >= 0:
#         mld = np.nan
#     else:
#         nthr_index = np.where(dens_diff > 0)[0]
#         if len(nthr_index) == 0:
#             naninds = np.where(np.isnan(dens_diff))[0]
#             if len(naninds) > 0:
#                 nanindex = naninds[0]
#             else:
#                 nanindex = len(depths)
#             mld = depths[nanindex-1]
#         else:
#             nind = nthr_index[0] + 1
#             mld = np.interp(0, dens_diff[:nind], depths[:nind])
#     return mld

def func_mld(dens_diff, depths):
    '''
    Function for calculate mld from density difference (den - den10 - 0.03) and depth
    Return mixed layer depth 
    '''
    if np.isnan(dens_diff[0]):
        mld = np.nan
    elif dens_diff[0] >= 0:
        mld = np.nan
    else:
        nthr_index = np.where(dens_diff > 0)[0]
        if len(nthr_index) == 0:
            naninds = np.where(np.isnan(dens_diff))[0]
            if len(naninds) > 0:
                nanindex = naninds[0]
            else:
                nanindex = len(depths)
            mld = depths[nanindex-1]
        else:
            nind = nthr_index[0] + 1
            mld = np.interp(0, dens_diff[nind-2:nind], depths[nind-2:nind])                
    return mld

# def xr_func_mld(dens):
#     '''
#     Function for parallel computing
#     '''
#     dens10 = dens.interp(lev = 10, method = 'linear')  # density at 10m
#     dens_diff = dens - dens10 - 0.03               # density differences 
#     mld = xr.apply_ufunc(
#         func_mld, 
#         dens_diff.chunk({"time":25, "lat":45, "lon":45}),  
#         dens_diff.lev, 
#         input_core_dims = [["lev"], ["lev"]], 
#         vectorize = True,
#         dask = "parallelized",
#         output_dtypes = [dens_diff.lev.dtype])
#     return mld

def xr_func_mld(dens):
    '''
    Function for parallel computing
    '''
    dens10 = dens.interp(lev = 10, method = 'linear')  # density at 10m
    dens_diff = dens - dens10 - 0.03               # density differences 
    mld = xr.apply_ufunc(
        func_mld, 
        dens_diff,#.chunk({"time":25, "x":30, "y":30}),  
        dens_diff.lev, 
        input_core_dims = [["lev"], ["lev"]], 
        vectorize = True,
        dask = "parallelized",
        output_dtypes = [dens_diff.lev.dtype],
    )
    return mld

In [ ]:
sep_mld = xr_func_mld(func_dens0(dataset_gr, start_year, end_year))

In [ ]:
# var='so'
# model='CESM2-WACCM'
# ofn = '/home/jovyan/tempfolder/' + model + '_' + var + 'depth_values' + start_year + end_year + '_np.txt'
# if os.path.exists(ofn):
#     dm=[]
#     with open(ofn, "r") as rf:
#         for line in rf:
#             dm.append(float(line.strip()))
#     values = np.array(dm)

# ofn = '/home/jovyan/tempfolder/' + model + '_' + var + 'depth_depths' + start_year + end_year + '_np.txt'
# if os.path.exists(ofn):
#     dm=[]
#     with open(ofn, "r") as rf:
#         for line in rf:
#             dm.append(float(line.strip()))
#     depths = np.array(dm)